In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import os


In [ ]:
IMAGE_SIZE = (100,100)
def load_images():
    image = []
    label = []
    image_path = []
    TRAIN_FOLDER_PATH = "C:/Users/pc/Documents/pythonProjects/OPSI2023/images/train" #ada sebanyak 3421 image
    TEST_FOLDER_PATH = "C:/Users/pc/Documents/pythonProjects/OPSI2023/images/test" #ada sebanyak 1871 image, total ada 5292 images
    for folder_name in os.listdir(TRAIN_FOLDER_PATH):
        for filename in os.listdir(os.path.join(TRAIN_FOLDER_PATH,folder_name)):
            print(filename)
            subfolder_path = os.path.join(folder_name,filename)
            img = cv2.imread(os.path.join(TRAIN_FOLDER_PATH,subfolder_path))
            img = cv2.resize(img,IMAGE_SIZE)
            image.append(img)
            label.append(folder_name)
            image_path.append(os.path.join(TRAIN_FOLDER_PATH,subfolder_path))

    for folder_name in os.listdir(TEST_FOLDER_PATH):
        for filename in os.listdir(os.path.join(TEST_FOLDER_PATH,folder_name)):
            print(filename)
            subfolder_path = os.path.join(folder_name,filename)
            img = cv2.imread(os.path.join(TEST_FOLDER_PATH,subfolder_path))
            img = cv2.resize(img,IMAGE_SIZE)
            image.append(img)
            label.append(folder_name)
            image_path.append(os.path.join(TEST_FOLDER_PATH,subfolder_path))
    image = np.array(image)
    return image,label, image_path

In [ ]:
image,label, image_path = load_images()

In [ ]:
print(len(image)) #banyaknya data gambar untuk train session
print(label)
print(len(label))
image.shape

In [ ]:
# i = 5000
# print(label[i])
# cv2.imshow('image',image[i])
# cv2.waitKey(0)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
label_encoder = OneHotEncoder()
new_label = np.reshape(label,(-1,1))
new_label = label_encoder.fit_transform(new_label).toarray()

In [ ]:
print(new_label)
print(len(new_label [0]))
print(new_label .shape)
# new_label = np.resize(new_label,(5292,10))
print(new_label.shape)
new_label[0]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
augmentation = ImageDataGenerator(
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(image,new_label,test_size=0.2,random_state=3)

In [ ]:
model_lenet5_optimized = keras.Sequential()
model_lenet5_optimized.add(keras.layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(100,100, 3)))
model_lenet5_optimized.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
model_lenet5_optimized.add(keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model_lenet5_optimized.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
model_lenet5_optimized.add(keras.layers.Flatten())

model_lenet5_optimized.add(keras.layers.Dense(units=120, activation='relu'))
model_lenet5_optimized.add(keras.layers.Dense(units=84, activation='relu'))
model_lenet5_optimized.add(keras.layers.Dense(units=6, activation='softmax'))

model_lenet5_optimized.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_lenet5_optimized.summary()

In [ ]:
history = model_lenet5_optimized.fit(augmentation.flow(x_train, y_train, batch_size=32),validation_data=(x_test,y_test),epochs=10)

In [ ]:
import matplotlib.pyplot as plt

# Plotting the loss curve
plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Pada Training Session', fontsize=10)

# Plotting the accuracy curve
plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Akurasi Pada Training Session',fontsize=10)

In [ ]:
model_lenet5_optimized.evaluate(x_test,y_test)

In [ ]:
model_lenet5_optimized.save('lenet5augmented.h5')

In [ ]:
prediction = model_lenet5_optimized.predict(x_test)
prediction

In [ ]:
jenis_kanker = ["Adenocarcinoma","Kanker Ganas","Kanker Jinak", "Sehat", "Sel Besar Karsinoma", "Sel Besar Skuamous"]
banyak_kesalahan = 0
for i in range(len(prediction)):
    if jenis_kanker[np.argmax(prediction[i])] != jenis_kanker[np.argmax(y_test[i])]:
        banyak_kesalahan += 1
        print("data ke :",  i+1)
        print("Prediction : " + jenis_kanker[np.argmax(prediction[i])])
        print("Kunci Jawaban: ",jenis_kanker[np.argmax(y_test[i])])
        print()
print(banyak_kesalahan)

In [ ]:
print(len(y_test))
banyak_kesalahan

In [ ]:
akurasi_training = model_lenet5_optimized.evaluate(x_train,y_train)[1]
akurasi_testing = model_lenet5_optimized.evaluate(x_test,y_test)[1]

In [ ]:
print("Akurasi training session :",akurasi_training)
print("Akurasi testing session :",akurasi_testing)